In [29]:
file_path='www.ssa.gov/www_ssa_gov_oact_ssir_SSI20_SingleYearTables_ssiSingleYearIndex_html.csv'

StatementMeta(smallpool, 21670, 30, Finished, Available)

In [40]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import pandas as pd
from io import StringIO
from pyspark.sql.functions import *
import requests
from bs4 import BeautifulSoup

StatementMeta(smallpool, 21670, 41, Finished, Available)

In [31]:
linked_service_name = 'bronze'
container_name = 'ingestion-meta'
account_name = 'usafactsbronze'
blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name)
blob_service_client = BlobServiceClient(account_url=f'https://{account_name}.blob.core.windows.net/', credential=blob_sas_token)
container_client = blob_service_client.get_container_client(container_name)

StatementMeta(smallpool, 21670, 32, Finished, Available)

In [32]:
def write_bronze(df,pdf_path):

    BLOB_ACCOUNT_NAME = 'usafactsbronze'
    LINKED_SERVICE_NAME = 'Bronze'
    BLOB_SAS_TOKEN = mssparkutils.credentials.getConnectionStringOrCreds(LINKED_SERVICE_NAME)
    blob_service_client = BlobServiceClient("https://{}.blob.core.windows.net".format(BLOB_ACCOUNT_NAME), credential=BLOB_SAS_TOKEN)
    container_name='bronze'

    ouput_file_path=pdf_path.replace("https://","").split('.')[:-1]
    ouput_file_path='.'.join(ouput_file_path)+'.csv'

    csv_data=df.to_csv(index=False)

    # Create a blob client for the new file
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=ouput_file_path)

    # Upload the CSV data to the blob
    blob_client.upload_blob(csv_data, overwrite=True)

    print("CSV file uploaded successfully:",ouput_file_path)
    

StatementMeta(smallpool, 21670, 33, Finished, Available)

In [33]:
def write_silver(df,ouput_file_path):    
    # silver storage access
    # Azure storage access info 
    blob_account_name = 'usafactssilver'
    blob_container_name = 'silver'
    linked_service_name = 'silver' 

    blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name) 

    ouput_file_path=ouput_file_path.replace('https://','')

    # Allow SPARK to access from Blob remotely
    wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, ouput_file_path)
    spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token)
    df.write.format('delta').mode('overwrite').option("overwriteSchema",True).option("path",wasbs_path).save()
    print('DELTA file uploaded successfully: ',ouput_file_path)


StatementMeta(smallpool, 21670, 34, Finished, Available)

In [34]:
blob_client = container_client.get_blob_client(file_path)
content = blob_client.download_blob().readall()

StatementMeta(smallpool, 21670, 35, Finished, Available)

In [89]:
# Convert bytes to string using StringIO
blob_string = str(content, 'utf-8')
blob_csv = StringIO(blob_string)

# Create Pandas DataFrame
df = pd.read_csv(blob_csv)

Footer=[]
# Create a list from the specified column
url_list = df.URL.tolist()
url_list=['https://www.ssa.gov/oact/ssir/SSI20/SingleYearTables/IV_A2.html']
for url in url_list:  
   
    # Sending HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the HTML content from the response
        html_content = response.text
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        url_links = [a for a in soup.find_all("div")]
        # print(url_links)
        header_text = soup.find('div', {'class': 'fs2 fw6 ta-c'}).get_text(strip=True)

        # # Split the text into lines to get individual headers
        headers = header_text.split('\n')

        # print(headers)

        # Find all div tags with class "fs2 fw6 ta-c"
        div_tags = soup.find_all('div', {'class': 'bt'})

        # print(div_tags)
        # Extract and print the text content within each div tag
        for div_tag in div_tags:
            header_text = div_tag.get_text(strip=True)
            footers = header_text.split('\n')
            # headers=' '.join(headers)
            for footer in footers:
                Footer.append(footer.strip())

            # print(header_text)
        div_tags = soup.find_all('div', {'class': 'pb2'})

        # print(div_tags)
        # Extract and print the text content within each div tag
        for div_tag in div_tags:
            header_text = div_tag.get_text(strip=True)
            footers = header_text.split('\n')
            # headers=' '.join(headers)
            for footer in footers:
                Footer.append(footer.strip())

            

StatementMeta(smallpool, 21670, 90, Finished, Available)

In [90]:
Footer

StatementMeta(smallpool, 21670, 91, Finished, Available)

['aIn this table the Social Security COLA is shown because',
 'it is used for the determination of the increase in the FBR each year. However, the actual',
 'application of these increases is different. For example, a newly eligible Social Security',
 'beneficiary in one year will generally receive a benefit in that year that is larger',
 'than the first year benefit for an equivalent worker who became eligible a year earlier by the',
 'increase in the average wage level in the economy. The average wage level has historically',
 'increased on average by about 1 percent relative to the annual increase in the CPI-W, on which the',
 'COLA is based.',
 'bIncreases prior to 1984 were effective for the',
 'payment due for June of the year. Increases shown for 1984 and later are effective',
 'for the payment due for December of the year.',
 'cIncreases prior to 1984 were effective for the',
 'payment due on July 1 of the year. Increases shown for 1984 and later are effective',
 'for the payme

In [35]:
# Convert bytes to string using StringIO
blob_string = str(content, 'utf-8')
blob_csv = StringIO(blob_string)

# Create Pandas DataFrame
df = pd.read_csv(blob_csv)


# Create a list from the specified column
url_list = df.URL.tolist()

for url in url_list:
    try:
        pandas_df=pd.read_html(url)
        pandas_df = pandas_df[1]

        pandas_df = pandas_df.dropna(axis=1)


        # write_bronze(pandas_df,url)

        spark_df=spark.createDataFrame(pandas_df)
        spark_df=spark_df.coalesce(1)
        columns = spark_df.columns
        new_columns = [col(c).alias(c.replace(" ", "_").replace(",", "_").replace(";", "_")
                            .replace("{", "_").replace("}", "_")
                            .replace("(", "_").replace(")", "_")
                            .replace("\n", "_").replace("\t", "_").replace("=", "_"))
                for c in spark_df.columns]

        # Apply the new column names to the DataFrame
        spark_df = spark_df.select(*new_columns)
        spark_df = spark_df.drop(*[col_name for col_name in spark_df.columns if spark_df.filter(col(col_name).isNotNull()).count() == 0])
        condition = ~((col(columns[0]) == 'Historical data:') | (col(columns[0]) == 'Projected:'))
        for column in columns:
            condition &= ~((col(column) == 'Historical data:') | (col(column) == 'Projected:'))

        spark_df = spark_df.filter(condition)
        spark_df=spark_df.withColumn('Header',lit(''))
        display(spark_df)
        # write_silver(spark_df,url)

    except:

        try:
            pandas_df=pd.read_html(url)
            pandas_df = pandas_df[1]
            pandas_df = pandas_df.dropna(axis=1,how='all')

            # write_bronze(pandas_df,url)

            spark_df=spark.createDataFrame(pandas_df)
            spark_df=spark_df.coalesce(1)
            columns = df.columns
            new_columns = [col(c).alias(c.replace(" ", "_").replace(",", "_").replace(";", "_")
                                .replace("{", "_").replace("}", "_")
                                .replace("(", "_").replace(")", "_")
                                .replace("\n", "_").replace("\t", "_").replace("=", "_"))
                    for c in spark_df.columns]

            # Apply the new column names to the DataFrame
            spark_df = spark_df.select(*new_columns)
            spark_df = spark_df.drop(*[col_name for col_name in spark_df.columns if spark_df.filter(col(col_name).isNotNull()).count() == 0])
            condition = ~((col(columns[0]) == 'Historical data:') | (col(columns[0]) == 'Projected:'))
            for column in columns:
                condition &= ~((col(column) == 'Historical data:') | (col(column) == 'Projected:'))
            spark_df = spark_df.filter(condition)
            display(spark_df)
            # write_silver(spark_df,url)

        except Exception as e:
            print(url,str(e))
    
    


StatementMeta(smallpool, 21670, 36, Cancelled, Waiting)

SynapseWidget(Synapse.DataFrame, 04a22358-5998-4a8c-87ec-0c6946cf0b95)

SynapseWidget(Synapse.DataFrame, ff8cfd46-d4f0-4d19-b3fd-4d2fa3bb1450)

SynapseWidget(Synapse.DataFrame, 24b2dc59-b0d4-478c-8d27-1e2a91924d88)

SynapseWidget(Synapse.DataFrame, 7717895e-8bfc-41f0-9fa2-c3a030444841)

SynapseWidget(Synapse.DataFrame, 66b276ed-463a-4a93-8361-78ed9dff7f67)